In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
from collections import Counter

%matplotlib inline

In [118]:
def process_file(file_id=100, nu_cut=1.0, beta=1.0, make_plot=False):

    peak_name = "map{:d}_100deg2_sl2.5_GSN_peaks.txt".format(file_id)
    peak_file = os.path.join("../data/", peak_name)
    all_peak_data = np.loadtxt(peak_file)

    # significance cut
    ii = all_peak_data[:,0]> nu_cut
    peak_data = all_peak_data[ii]

    peak_pos_file = peak_file.replace("map", "pos")
    np.savetxt(peak_pos_file, peak_data[:,1:])

    beta = 1.0
    out_filename = "../data/beta_{:.1f}_nu_{:.1f}_{}".format(beta, nu_cut, peak_name)


    skel = "/Users/forero/Applications/ngl-beta/build/binsrc/./getNeighborGraph "

    if not os.path.exists(out_filename):
        cmd = "{} -d 2 -i {} -b {} -o > {}".format(skel, peak_pos_file, beta, out_filename)
        print(cmd)
        os.system(cmd)
    
    # read in the pairs
    point_data = np.loadtxt(peak_pos_file)
    beta_data = np.int_(np.loadtxt(out_filename))

    # remove the duplicate points using the fact that the first column is ordered in ngl-beta
    ii = beta_data[:,0]<beta_data[:,1]
    beta_data = beta_data[ii]

    n_points = len(point_data)
    print(n_points)

    # Flattend and count links
    beta_data_flat = beta_data.flatten()
    beta_link_count = Counter(Counter(beta_data_flat).values())

    # Count how many points have zero links
    unique_beta_id = len(set(beta_data_flat))
    if (n_points - unique_beta_id)>0:
        beta_link_count[0] = n_points - unique_beta_id



    assert np.sum(list(beta_link_count.values()))==n_points
    # compute probability array
    proba = []
    total_link = np.sum(list(beta_link_count.values()))
    for k in beta_link_count:
        #print(k, beta_link_count[k]/total_link)
        proba.append(beta_link_count[k]/total_link)
    proba = np.array(proba)
    print(proba, proba.sum())
    entropy = np.sum(-proba*np.log(proba)/np.log10(2))
    print('Entropy', entropy)
    mean_conections = len(beta_data)/len(point_data)
    print('Mean connections', mean_conections)

    # compute the distances
    delta_x  = point_data[beta_data[:,0],0] - point_data[beta_data[:,1],0]
    delta_y = point_data[beta_data[:,0],1] - point_data[beta_data[:,1],1]
    distance = np.sqrt(delta_x**2 + delta_y**2)
    #_ = plt.hist(distance)
    normed_mean_distance = np.mean(distance)/np.std(distance)


    # plot the skeleton
    if make_plot:
        title = r"$\beta={:.1f}$, $\nu>{:.1f}$, Entropy={:.2f} Sh, $\langle l\rangle / \sigma_l = {:.2f}$, $\kappa={:1.f}$".format(
            beta, nu_cut, entropy, normed_mean_distance, mean_connections)
        plt.figure(figsize=(8,8))
        for p in beta_data:
            #print(p)
            plt.plot(point_data[p,0], point_data[p,1], c='black')
        #plt.scatter(all_peak_data[:,1], all_peak_data[:,2],c=all_peak_data[:,0], alpha=0.4)
        plt.scatter(peak_data[:,1], peak_data[:,2],c='black')
        plt.axis('scaled')
        plt.title(title)

In [83]:
np.log10(4) / np.log10(2)

2.0